In [1]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from optimum.bettertransformer import BetterTransformer
import torch

c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

torch.cuda.empty_cache()

Torch version: 2.1.1
Is CUDA enabled? True


In [3]:
dataset = load_dataset("financial_phrasebank", 'sentences_50agree')

neutrals = dataset.filter(lambda example: example["label"] == 1)
positives = dataset.filter(lambda example: example["label"] == 2)
negatives = dataset.filter(lambda example: example["label"] == 0)

short_positives = positives['train'].select(range(len(negatives['train'])))

print(short_positives)

downsized_dataset = concatenate_datasets([short_positives, negatives['train']])

print(downsized_dataset)

Found cached dataset financial_phrasebank (C:/Users/Gusso/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|██████████| 1/1 [00:00<00:00, 500.93it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-4069f8bdc2072a81.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-255302c37e6b94c9.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-913aca77f5bc0114.arrow


Dataset({
    features: ['sentence', 'label'],
    num_rows: 604
})
Dataset({
    features: ['sentence', 'label'],
    num_rows: 1208
})


In [4]:
def replace_values(example):
    # Replace values in the label column
    if "label" in example:
        example["label"] = 1 if example["label"] == 2 else example["label"]
    return example

# Use the map function to apply the custom mapping
downsized_dataset = downsized_dataset.map(replace_values)
downsized_dataset = downsized_dataset.train_test_split(test_size=0.1, shuffle=True)
print(downsized_dataset)

Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-9ab259d81c7af6d1.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1087
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 121
    })
})


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True)

tokenized_datasets = downsized_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = downsized_dataset['test'].map(tokenize_function, batched=True)

In [2]:
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased")

model = model.to(0)

print(model.get_memory_footprint())

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=2, per_device_eval_batch_size=2, gradient_accumulation_steps=8, gradient_checkpointing=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1340583944


In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)


In [8]:
trainer.train()

  0%|          | 0/204 [00:00<?, ?it/s]c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
                                                
 33%|███▎      | 68/204 [02:22<04:30,  1.99s/it]c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False.

{'eval_loss': 0.1745164543390274, 'eval_accuracy': 0.9504132231404959, 'eval_runtime': 4.0585, 'eval_samples_per_second': 29.814, 'eval_steps_per_second': 15.03, 'epoch': 1.0}


                                                 
 67%|██████▋   | 136/204 [04:48<02:20,  2.07s/it]c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'eval_loss': 0.17824231088161469, 'eval_accuracy': 0.9669421487603306, 'eval_runtime': 4.1735, 'eval_samples_per_second': 28.992, 'eval_steps_per_second': 14.616, 'epoch': 2.0}


                                                 
100%|██████████| 204/204 [07:13<00:00,  2.13s/it]

{'eval_loss': 0.18680688738822937, 'eval_accuracy': 0.9669421487603306, 'eval_runtime': 3.828, 'eval_samples_per_second': 31.609, 'eval_steps_per_second': 15.935, 'epoch': 3.0}
{'train_runtime': 433.7412, 'train_samples_per_second': 7.518, 'train_steps_per_second': 0.47, 'train_loss': 0.1258710786408069, 'epoch': 3.0}


TrainOutput(global_step=204, training_loss=0.1258710786408069, metrics={'train_runtime': 433.7412, 'train_samples_per_second': 7.518, 'train_steps_per_second': 0.47, 'train_loss': 0.1258710786408069, 'epoch': 3.0})

In [9]:
model = BetterTransformer.transform(model)

trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [10]:
validation_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')

def filter_function(example):
    return example["label"] != 2

# Use the filter method to remove records
filtered_dataset = validation_dataset.filter(filter_function)

def tokenize_function2(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

tokenized_datasets_validation = filtered_dataset.map(tokenize_function2, batched=True)

# Print the modified dataset
print(dataset)

Found cached dataset csv (C:/Users/Gusso/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 998.88it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b2d98c45395bc49c.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-98a6e9e55fa382e5.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cach

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})


In [11]:
trainer2.evaluate(eval_dataset=tokenized_datasets_validation['validation'])

c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\optimum\bettertransformer\models\encoder_models.py:301: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, ~attention_mask)
100%|██████████| 411/411 [00:09<00:00, 41.46it/s]


{'eval_loss': 0.5935923457145691,
 'eval_accuracy': 0.8832116788321168,
 'eval_runtime': 9.9895,
 'eval_samples_per_second': 82.286,
 'eval_steps_per_second': 41.143}